In [1]:
!pip install PyMuPDF
!pip install pytesseract
!pip install opencv-python-headless
!pip install pillow
!pip install pandas
!pip install pytesseract

import pytesseract
#!wget https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/
pytesseract.pytesseract.tesseract_cmd = "C:/Tesseract-OCR/tesseract.exe"
#!apt-get install tesseract-ocr


#!pip install --upgrade pandas scikit-learn transformers
#!pip install pandas --user --upgrade
#!pip install --upgrade scipy
#!pip install --upgrade numpy

#!pip install --upgrade transformers
#!pip install nlpaug
#!pip install --user --upgrade numpy


C:\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
import fitz
import pandas as pd
import pytesseract
import re
from PIL import Image
import time

#!wget https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/
pytesseract.pytesseract.tesseract_cmd = "C:/Tesseract-OCR/tesseract.exe"
#!apt-get install tesseract-ocr



# OCR - EXTRACT TEXT FROM A PFD

In [7]:
def extract_text_from_pdf(pdf_file_path,start_page,end_page):

    # Abrir o arquivo PDF
    pdf_document = fitz.open(pdf_file_path)
    
    
    # Configurar o caminho para o executável do Tesseract OCR
    #pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

    # Obter o número total de páginas no PDF
    total_pages = pdf_document.page_count

    # Inicializar variável para o tempo total
    total_time = 0

    # Lista para armazenar os resultados das páginas
    results = []

    # Imprimir informações sobre o início do processamento
    print(f'Iniciando processamento das páginas {start_page} a {end_page} de um total de {total_pages} páginas...')

    # Iniciar contagem de tempo
    start_total_time = time.time()

    # Processar cada página do PDF
    for page_number in range(start_page, min(end_page, total_pages)):
        page = pdf_document.load_page(page_number)

        # Extrair imagem da página como uma matriz numpy
        pixmap = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
        image_bytes = pixmap.samples

        # Converter bytes em imagem usando o módulo Pillow (PIL)
        image = Image.frombytes("RGB", [pixmap.width, pixmap.height], image_bytes)

        # Iniciar contagem de tempo para a página atual
        start_time = time.time()

        # Extrair texto da imagem usando pytesseract
        extracted_text = pytesseract.image_to_string(image, lang='por')
        end_time = time.time()

        # Calcular o tempo de processamento para a página atual
        page_time = end_time - start_time

        # Armazenar os resultados da página atual na lista de resultados
        results.append({'Número da Página': page_number + 1,
                        'Texto da Página': extracted_text,
                        'Tempo de Processamento (s)': page_time})

        # Imprimir o progresso do processamento
        progress = (page_number + 1) / (min(end_page, total_pages) - start_page) * 100
        print(f'Página {page_number + 1}/{min(end_page, total_pages)} processada ({progress:.2f}%)')

    # Parar a contagem de tempo total
    end_total_time = time.time()

    # Calcular o tempo total de processamento
    total_time = end_total_time - start_total_time

    # Fechar o arquivo PDF
    pdf_document.close()

    # Criar um DataFrame a partir dos resultados das páginas
    df = pd.DataFrame(results)


    # Imprimir o tempo total de processamento
    print(f'Tempo total de processamento: {total_time:.2f} segundos')
    return df



In [49]:
#pdf_file_path = "pdfs_treino_roted_1000.pdf"
#df_treino = extract_text_from_pdf(pdf_file_path,0,1000)
#df_treino.to_csv("ocr-lab-eng-software-treino.csv")
df_treino = pd.read_csv("ocr-lab-eng-software-treino.csv")
df_treino

,Unnamed: 0,Número da Página,Texto da Página,Tempo de Processamento (s)
0,0,1,"06:21:42\n\nHOTEI. SÃO GERAIDO 4 o,\nT. DE ALM...",3.163012
1,1,2,EN\n,0.930657
2,2,3,pe sEasas cosnoces 2.9.50.00 7 RESERVADO\n\np4...,3.422160
3,3,4,NaN,0.928966
4,4,5,06:10:34 GerHotel 1.9.50.001 ESERVAD 24/06/202...,3.036623
...,...,...,...,...
995,995,996,"“Dyetoria de “de Planejam ento, Orçamento, :\n...",1.747922
996,996,997,Es - Prefeitura do Município de Maringá - PR N...,2.733784
997,997,998,NaN,1.004460
998,998,999,"NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe,\n\n...",2.992928


In [9]:
df_treino[0:779]

,Unnamed: 0,Número da Página,Texto da Página,Tempo de Processamento (s)
0,0,1,"06:21:42\n\nHOTEI. SÃO GERAIDO 4 o,\nT. DE ALM...",3.163012
1,1,2,EN\n,0.930657
2,2,3,pe sEasas cosnoces 2.9.50.00 7 RESERVADO\n\np4...,3.422160
3,3,4,NaN,0.928966
4,4,5,06:10:34 GerHotel 1.9.50.001 ESERVAD 24/06/202...,3.036623
...,...,...,...,...
774,774,775,"ET,\n\n[ emitem - -—— - ES pra\n\nGate (Gourme...",1.967096
775,775,776,Numero da Nota\n\nPREFEITURA DA CIDADE DO RIO ...,2.938792
776,776,777,APRE So\nGate Gourmet Ltda bandido fl nd NOTA ...,2.124802
777,777,778,NF-e\nNº 000.017.008\nSÉRIE 001\n\neee\n\nDocu...,3.930316


In [50]:
#pdf_file_path = "pdfs_teste_manual_rotated_112.pdf"
#df_teste = extract_text_from_pdf(pdf_file_path,0,112)
#df_teste.to_csv("ocr-lab-eng-software-teste.csv")
df_teste = pd.read_csv("ocr-lab-eng-software-teste.csv")

In [51]:
df_teste = pd.concat([df_treino[780:1000], df_teste])
df_treino = df_treino[0:779]


In [52]:
df_teste

,Unnamed: 0,Número da Página,Texto da Página,Tempo de Processamento (s)
780,780,781,NaN,0.740475
781,781,782,RESERVADO\n\n2.2 ALIMENTAÇÃO pr\n\nForam reali...,5.915373
782,782,783,NaN,0.775917
783,783,784,OdVAHISTA\n\n0S'889'9 SH\n\ncc/80/80 9P\n0S'88...,4.138452
784,784,785,E\n\n[a d\n\ncurto\n\nRS e\n|| a F\n\n[Va\n\n,1.778829
...,...,...,...,...
107,107,108,NaN,1.493463
108,108,109,LAGHETTO STILO BARRA RIO\nExtrato de Conta\n\n...,3.661777
109,109,110,"Er\n\nAu zo 656/(UZo 052/\n\n' 1. / 1 ge dg,\n...",4.218797
110,110,111,a E ao\nSECRETARIA MUNICIPAL DE FAZENDA\nNOTA ...,4.803314


# BUSCA PALAVRAS CHAVE

In [70]:
# Função para verificar se uma string contém qualquer uma das palavras-chave
def contains_keywords(text, keywords):
    # Converter a string para letras minúsculas 
    #try:
        #text_cleaned = text.lower()
    #except:
        #text = 'nan'

    text_cleaned = text.lower()

    
    # Palavras-chave como um conjunto para pesquisa otimizada
    keyword_set = set(keywords)

    # Contador para rastrear quantas palavras-chave foram encontradas
    keyword_count = 0

    # Lista para rastrear as palavras encontradas
    found_keywords = []

    # Verificar se a string contém qualquer uma das palavras-chave usando pesquisa de conjunto
    for keyword in text_cleaned.split():
        if keyword in keyword_set:
            keyword_count += 1
            found_keywords.append(keyword)

    return keyword_count, found_keywords

# Função para extrair valores numéricos da página
def extract_numerical_values(text):
    # Usar expressão regular para encontrar valores numéricos (incluindo decimais)
    try:
        numerical_values = re.findall(r'\b\d+\.\d+|\b\d+\,\d+|\d+\b', text)
        return numerical_values
    except:
        return []

def extract_money_values(text):
    try:    
        money_values = re.findall(r'[0-9]*\.?[0-9]*\.?[0-9]{1,3}\,[0-9]{2}', text)
        return money_values
    except:
        return []    
    
def extract_dates(text):
    try:    
        date_pattern = r'\d{2}/\d{2}/\d{4}'
        dates = re.findall(date_pattern, text.astype(str))
        return dates
    except:
        return []        

def find_date(text):
    try:    
        dates = re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}', text)[0] if re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}', text) != [] else None
        return dates
    except:
        return []    
    
def find_cnpj(text):
    try:   
        re.findall(r'[0-9]{2}.[0-9]{3}.[0-9]{3}/[0-9]{4}-[0-9]{2}', text)[0] if re.findall(r'[0-9]{2}.[0-9]{3}.[0-9]{3}/[0-9]{4}-[0-9]{2}', text) != [] else None,
    except:
        return []      
    
def find_mode(extract_money_values):
    try:
        return [statistics.mode(extract_money_values)]
        
    except:
        return []

def search_text_from_df(df, n_rows_to_search, positive_keywords,negative_keywords):
    

    # Inicializar variável para o tempo total
    total_time = 0

    # Lista para armazenar os resultados das páginas
    searched_list = []

    # Iniciar contagem de tempo
    start_total_time = time.time()

    for page_number in range(0, n_rows_to_search):

        page = df.iloc[page_number,2]#pdf_document.load_page(page_number) #mocado, precisa encontrar a coluna de texto sozinho
        # Iniciar contagem de tempo para a página atual
        start_time = time.time()

        # Extrair texto da imagem usando pytesseract
        extracted_text = page#pytesseract.image_to_string(image, lang='por')

        try:
            extracted_text = extracted_text.lower()
        except:
            text = 'nan'
            extracted_text = text.lower()

    

        negative_keyword_count, negative_found_keywords = contains_keywords(extracted_text, negative_keywords)
        negative_found_keywords = set(negative_found_keywords)
        negative_keyword_count = len(negative_found_keywords)

        # Contar quantas palavras-chave aparecem no texto da página e obter a lista das palavras encontradas
        keyword_count, found_keywords = contains_keywords(extracted_text, positive_keywords)
        found_keywords = set(found_keywords) - set(negative_found_keywords)
        keyword_count = len(found_keywords) - negative_keyword_count
        
        is_boleto_extrato_recibo = 'extrato' in negative_found_keywords or 'boleto' in negative_found_keywords or 'recibo' in negative_found_keywords
        
            # Armazenar os resultados da página atual na lista de resultados
        searched_list.append({'n_pagina': page_number,
                        'n_palavras_encontradas': keyword_count,
                        'palavras_encontradas_nf': found_keywords,
                        'palavras_encontradas_nao_nf': negative_found_keywords,
                        'texto_encontrado': extracted_text,
                        'boleto_extrato_recibo':is_boleto_extrato_recibo,
                        'valores_numericos_encontrados': extract_numerical_values(extracted_text),
                       # 'datas_encontradas': extract_dates(extracted_text),
                        "datas_encontradas": find_date(extracted_text),
                        "cnpj_encontrado": find_cnpj(extracted_text),
                        "valores_monetarios_encontrados": extract_money_values(extracted_text),
                        "maior_valor_nota": max(extract_money_values(extracted_text), default=0),
                         "valor_mais_comum_nota": find_mode(extract_money_values(extracted_text))})


    return pd.DataFrame(searched_list)

In [71]:
# Palavras-chave comuns em notas fiscais brasileiras (separadas)
positive_keywords = ["nota","danfe","emissao","chave","acesso","eletronica", "fiscal", "cnpj", "cpf", "data", "valor", "total", "descrição", "quantidade", "preço", "unitário",
            "subtotal", "impostos", "desconto", "frete", "nome", "razão", "social", "endereço", "telefone",
            "inscrição", "estadual", "municipal", "número", "série", "observações",
         "NFS-e", "fatura", "serviço", "valor", "total","cálculo","alíquota","eletrônica", "nf", "nf-e","fiscal"]

negative_keywords= ["rps","provisory", "reservado","provisorio", "recibo", "cupom", "extrato", "receipt","visa","safrapay","boleto","via","cliente"]

df_treino_keywords = search_text_from_df(df_treino, len(df_treino),positive_keywords,negative_keywords)
df_treino_keywords

,n_pagina,n_palavras_encontradas,palavras_encontradas_nf,palavras_encontradas_nao_nf,texto_encontrado,boleto_extrato_recibo,valores_numericos_encontrados,datas_encontradas,cnpj_encontrado,valores_monetarios_encontrados,maior_valor_nota,valor_mais_comum_nota
0,0,3,"{nome, data, valor, total}",{extrato},"06:21:42\n\nhotei. são geraido 4 o,\nt. de alm...",True,"[06, 21, 42, 4, 30, 59650000, 5, 8433312628, 8...",20/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[]
1,1,0,{},{},en\n,False,[],None,None,[],0,[]
2,2,2,"{nome, data, valor, total}","{reservado, extrato}",pe seasas cosnoces 2.9.50.00 7 reservado\n\np4...,True,"[2.9, 50.00, 7, 4, 21, 30, 59650000, 843331262...",20/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[]
3,3,0,{},{},nan,False,[],None,None,[],0,[]
4,4,3,"{nome, data, valor, total}",{extrato},06:10:34 gerhotel 1.9.50.001 eservad 24/06/202...,True,"[06, 10, 34, 1.9, 50.001, 24, 06, 2021, 30, 59...",24/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
774,774,1,"{nota, total}",{cliente},"et,\n\n[ emitem - -—— - es pra\n\ngate (gourme...",False,"[19884, 21941570, 103, 25, 07, 22, 35, 21, 239...",None,None,"[69,01, 0.040,74, 4,41, 500,00, 32,00, 32,00]","69,01",[]
775,775,11,"{cálculo, desconto, alíquota, data, fiscal, no...",{rps},numero da nota\n\nprefeitura da cidade do rio ...,False,"[25, 07, 22, 14, 22, 09, 69.012, 656, 0011, 00...",15/10/2005,None,"[6,00, 0,00, 0,00, 0,00, 0,00, 35,00, 11,05, 5...","6,00",[]
776,776,0,"{nota, total}","{cliente, via}",apre so\ngate gourmet ltda bandido fl nd nota ...,False,"[19833, 21941570, 103, 7, 25, 07, 22, 55, 21, ...",None,None,"[3.803,18, 243,40, 243,40]","3.803,18",[]
777,777,22,"{cálculo, desconto, estadual, descrição, quant...",{reservado},nf-e\nnº 000.017.008\nsérie 001\n\neee\n\ndocu...,False,"[000.017, 008, 001, 3322, 0769, 0126, 5600, 11...",25/07/2022,None,"[3.803,18, 3.803,18, 3.803,18, 3.803,18, 3.803...","7,00",[]


In [55]:
df_teste_keywords = search_text_from_df(df_teste, len(df_teste),positive_keywords,negative_keywords)
df_teste_keywords

,n_pagina,n_palavras_encontradas,palavras_encontradas_nf,palavras_encontradas_nao_nf,texto_encontrado,boleto_extrato_recibo,valores_numericos_encontrados,datas_encontradas,cnpj_encontrado,valores_monetarios_encontrados,maior_valor_nota,valor_mais_comum_nota
0,0,0,{},{},nan,False,[],None,None,[],0,[]
1,1,4,"{data, preço, total, unitário, cnpj}",{reservado},reservado\n\n2.2 alimentação pr\n\nforam reali...,False,"[2.2, 147, 148, 03, 03, 2022, 123, 04, 2022, 1...",None,None,"[38,00]","38,00",[]
2,2,0,{},{},nan,False,[],None,None,[],0,[]
3,3,0,{},{},odvahista\n\n0s'889'9 sh\n\ncc/80/80 9p\n0s'88...,False,"[889, 9, 80, 80, 889, 9, 80, 0100, 0, 9, 0, 0,...",None,None,[],0,[]
4,4,0,{},{},e\n\n[a d\n\ncurto\n\nrs e\n|| a f\n\n[va\n\n,False,[],None,None,[],0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
327,327,0,{},{},nan,False,[],None,None,[],0,[]
328,328,3,"{descrição, nome, data, valor, total}","{extrato, safrapay}",laghetto stilo barra rio\nextrato de conta\n\n...,True,"[1215, 118, 14206828, 00394411000109, 2, 2, 0,...",25/02/2019,None,"[290,00, 290,00, 290,00, 580,00, 290,00, 870,0...","870,00",[]
329,329,-4,{},"{safrapay, visa, cliente, via}","er\n\nau zo 656/(uzo 052/\n\n' 1. / 1 ge dg,\n...",False,"[656, 052, 1, 1, 79, 060, 054, 19098146, 29082...",25/82/2019,None,[],0,[]
330,330,12,"{cálculo, desconto, alíquota, nome, fiscal, da...",{rps},a e ao\nsecretaria municipal de fazenda\nnota ...,False,"[16, 56, 52, 29.060, 059, 0001, 57, 1.104, 661...",18/02/2019,None,"[2030,00, 2.030,00, 0,00, 0,00, 2.030,00, 5,00...","5,00",[]


In [28]:
#pdf_file_path = "pdfs_test_roted_automatico_246.pdf"
#df_oot = extract_text_from_pdf(pdf_file_path,0,246)
#df_oot.to_csv("ocr-lab-eng-software-df_oot.csv")
#df_oot_keyword=search_text_from_df(df_oot, len(df_oot),positive_keywords,negative_keywords)

In [29]:
#df_oot_keyword=search_text_from_df(df_oot, len(df_oot),positive_keywords,negative_keywords)

# TOKENIZER

In [14]:
def bert_tokenizer(df,text_column):
    from transformers import BertTokenizer
    from sklearn.feature_extraction.text import TfidfVectorizer

    # Carregar o tokenizer BERT
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenizar os textos usando TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[text_column].astype(str).tolist())
    
    return tfidf_matrix


In [15]:
treino_tfidf = bert_tokenizer(df_treino,'Texto da Página')
teste_tfidf = bert_tokenizer(df_teste,'Texto da Página')
print(treino_tfidf.shape)
print(teste_tfidf.shape)

(779, 19244)
(112, 3734)


In [ ]:
treino_tfidf

# LABEL


In [45]:
import numpy as np

In [90]:
df_treino_keywords['is_nf'] = np.where(((df_treino_keywords.n_palavras_encontradas >= 6) & (df_treino_keywords.boleto_extrato_recibo == False)), 1, 0)
#df_oot_keyword['is_nf'] = np.where(((df_oot_keyword.n_palavras_encontradas >= 6) & (df_oot_keyword.boleto_extrato_recibo == False)), 1, 0)
df_teste_keywords['is_nf'] = np.where(((df_teste_keywords.n_palavras_encontradas >= 6) & (df_teste_keywords.boleto_extrato_recibo == False)), 1, 0)

#df_teste_keywords['is_nf'] = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1]

In [91]:
df_treino_keywords.is_nf.value_counts()

is_nf
0    599
1    180
Name: count, dtype: int64

In [92]:
len(df_treino_keywords)

779

In [93]:
df_teste_keywords.is_nf.value_counts()

is_nf
0    215
1    117
Name: count, dtype: int64

In [73]:
len(df_teste_keywords)

332

# MODEL TRAINING

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report


def train_multiples_ml_models(train_df, test_df, label, column_text_train, column_text_test, models = [
    ("SVM", SVC(kernel='linear', C=1.0,probability=True)),
    ("Random Forest", RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)),
    ("XGBoost", XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=10, random_state=42)),
    ("Logistic Regression", LogisticRegression(C=0.1, max_iter=1000, random_state=42)),
    ("Gradient Boosting", GradientBoostingClassifier(n_estimators=10, max_depth=3, learning_rate=0.1, random_state=42))
]):
    train_df['label'] = train_df[label].astype(int)
    train_df['sentence'] = train_df[column_text_train]
    test_df['label'] = test_df[label].astype(int)
    test_df['sentence'] = test_df[column_text_test]

    # Carregar o tokenizer BERT
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Converter rótulos para tensores do PyTorch
    train_labels = torch.tensor(train_df['label'].tolist())
    test_labels = torch.tensor(test_df['label'].tolist())

    # Tokenizar os textos usando TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    train_tfidf = tfidf_vectorizer.fit_transform(train_df['sentence'])
    test_tfidf = tfidf_vectorizer.transform(test_df['sentence'])


    # Inicializar listas para armazenar métricas
    model_names = []
    train_accuracies = []
    train_precisions = []
    train_recalls = []
    train_f1_scores = []
    test_accuracies = []
    test_precisions = []
    test_recalls = []
    test_f1_scores = []
    class_report_dict = {'model':[],'class_report':[]}
    results_ml_df = {}
    
    report_df_total = pd.DataFrame()
    # Converter rótulos para tensores do PyTorch
    train_labels = torch.tensor(train_df['label'].tolist())
    test_labels = torch.tensor(test_df['label'].tolist())
    
    for model_name, model in models:
        # Treinar e avaliar modelos
        model.fit(train_tfidf, train_labels)
        train_predictions = model.predict(train_tfidf)
        test_predictions = model.predict(test_tfidf)
        test_proba = model.predict_proba(test_tfidf)

        # Calcular métricas de treinamento
        train_accuracy = accuracy_score(train_labels, train_predictions)
        train_precision = precision_score(train_labels, train_predictions)
        train_recall = recall_score(train_labels, train_predictions)
        train_f1 = f1_score(train_labels, train_predictions)

        # Calcular métricas de teste
        test_accuracy = accuracy_score(test_labels, test_predictions)
        test_precision = precision_score(test_labels, test_predictions)
        test_recall = recall_score(test_labels, test_predictions)
        test_f1 = f1_score(test_labels, test_predictions)

        # Adicionar métricas às listas
        model_names.append(model_name)
        train_accuracies.append(train_accuracy)
        train_precisions.append(train_precision)
        train_recalls.append(train_recall)
        train_f1_scores.append(train_f1)
        test_accuracies.append(test_accuracy)
        test_precisions.append(test_precision)
        test_recalls.append(test_recall)
        test_f1_scores.append(test_f1)
    
        class_report_dict['model'].append(model_name)
        class_report_dict['class_report'].append(classification_report(test_labels, test_predictions))
        
        
        report_df = pd.DataFrame(classification_report(test_labels, test_predictions,output_dict=True))
        report_df['model'] = [model_name]*len(report_df)
        report_df_total = pd.concat([report_df_total, report_df])
        confiabilidade = []
        for i in test_proba:
            confiabilidade.append(abs(i[0]-i[1]))
    
        # Criar um DataFrame com os resultados
        results_ml_df.update({model_name:pd.DataFrame({'Página': test_df['n_pagina'], 'Texto Extraído': test_df['sentence'], 'É uma nota?': test_predictions, 'Confiabilidade':confiabilidade, 'is_nf_input':test_labels, 'Maior valor da nota':test_df.maior_valor_nota, 'Data mais comum':test_df.datas_encontradas})})



    # Criar um DataFrame com as métricas
    metrics_df = pd.DataFrame({
        "Modelo": model_names,
        "Acurácia Treino": train_accuracies,
        "Precisão Treino": train_precisions,
        "Recall Treino": train_recalls,
        "F1-Score Treino": train_f1_scores,
        "Acurácia Teste": test_accuracies,
        "Precisão Teste": test_precisions,
        "Recall Teste": test_recalls,
        "F1-Score Teste": test_f1_scores
    })
    

    # Exibir a tabela de métricas
    return metrics_df,report_df_total,results_ml_df

C:\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
metrics_df,class_report,results_ml_df=train_multiples_ml_models(df_treino_keywords, df_teste_keywords, label='is_nf', column_text_train='texto_encontrado', column_text_test='texto_encontrado')

NameError: name 'df_treino_keywords' is not defined

In [120]:
class_report

,0,1,accuracy,macro avg,weighted avg,model
precision,0.985294,0.890625,0.948795,0.937960,0.951932,SVM
recall,0.934884,0.974359,0.948795,0.954621,0.948795,SVM
f1-score,0.959427,0.930612,0.948795,0.945020,0.949273,SVM
support,215.000000,117.000000,0.948795,332.000000,332.000000,SVM
precision,0.980769,0.911290,0.954819,0.946030,0.956284,Random Forest
recall,0.948837,0.965812,0.954819,0.957325,0.954819,Random Forest
f1-score,0.964539,0.937759,0.954819,0.951149,0.955102,Random Forest
support,215.000000,117.000000,0.954819,332.000000,332.000000,Random Forest
precision,0.965854,0.866142,0.927711,0.915998,0.930714,XGBoost
recall,0.920930,0.940171,0.927711,0.930551,0.927711,XGBoost


In [121]:
metrics_df

,Modelo,Acurácia Treino,Precisão Treino,Recall Treino,F1-Score Treino,Acurácia Teste,Precisão Teste,Recall Teste,F1-Score Teste
0,SVM,0.982028,0.961111,0.961111,0.961111,0.948795,0.890625,0.974359,0.930612
1,Random Forest,0.997433,0.994444,0.994444,0.994444,0.954819,0.911290,0.965812,0.937759
2,XGBoost,0.971759,0.929348,0.950000,0.939560,0.927711,0.866142,0.940171,0.901639
3,Logistic Regression,0.853659,1.000000,0.366667,0.536585,0.816265,0.966667,0.495726,0.655367
4,Gradient Boosting,0.962773,0.980892,0.855556,0.913947,0.894578,0.918367,0.769231,0.837209


In [62]:
results_ml_df['SVM']

,Página,Texto Extraído,É uma nota?,Confiabilidade,is_nf_input,Maior valor da nota,Valor mais comum da nota
0,0,nan,0,0.978839,0,0,[]
1,1,reservado\n\n2.2 alimentação pr\n\nforam reali...,0,0.976321,0,"38,00",[]
2,2,nan,0,0.978839,0,0,[]
3,3,odvahista\n\n0s'889'9 sh\n\ncc/80/80 9p\n0s'88...,0,0.919093,0,0,[]
4,4,e\n\n[a d\n\ncurto\n\nrs e\n|| a f\n\n[va\n\n,0,0.932154,0,0,[]
...,...,...,...,...,...,...,...
327,327,nan,0,0.978839,0,0,[]
328,328,laghetto stilo barra rio\nextrato de conta\n\n...,0,0.946708,0,"870,00",[]
329,329,"er\n\nau zo 656/(uzo 052/\n\n' 1. / 1 ge dg,\n...",0,0.977018,0,0,[]
330,330,a e ao\nsecretaria municipal de fazenda\nnota ...,1,0.993982,1,"5,00",[]


In [122]:
results_ml_df['SVM'].to_csv("resultados_tcc2_2.csv")

In [64]:
df_treino_keywords

,n_pagina,n_palavras_encontradas,palavras_encontradas_nf,palavras_encontradas_nao_nf,texto_encontrado,boleto_extrato_recibo,valores_numericos_encontrados,datas_encontradas,cnpj_encontrado,valores_monetarios_encontrados,maior_valor_nota,valor_mais_comum_nota,is_nf,label,sentence
0,0,3,"{nome, data, valor, total}",{extrato},"06:21:42\n\nhotei. são geraido 4 o,\nt. de alm...",True,"[06, 21, 42, 4, 30, 59650000, 5, 8433312628, 8...",20/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[],0,0,"06:21:42\n\nhotei. são geraido 4 o,\nt. de alm..."
1,1,0,{},{},en\n,False,[],None,None,[],0,[],0,0,en\n
2,2,2,"{nome, data, valor, total}","{reservado, extrato}",pe seasas cosnoces 2.9.50.00 7 reservado\n\np4...,True,"[2.9, 50.00, 7, 4, 21, 30, 59650000, 843331262...",20/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[],0,0,pe seasas cosnoces 2.9.50.00 7 reservado\n\np4...
3,3,0,{},{},nan,False,[],None,None,[],0,[],0,0,nan
4,4,3,"{nome, data, valor, total}",{extrato},06:10:34 gerhotel 1.9.50.001 eservad 24/06/202...,True,"[06, 10, 34, 1.9, 50.001, 24, 06, 2021, 30, 59...",24/06/2021,None,"[100,00, 100,00, 100,00, 100,00, 400,00, 400,0...","400,00",[],0,0,06:10:34 gerhotel 1.9.50.001 eservad 24/06/202...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,774,1,"{nota, total}",{cliente},"et,\n\n[ emitem - -—— - es pra\n\ngate (gourme...",False,"[19884, 21941570, 103, 25, 07, 22, 35, 21, 239...",None,None,"[69,01, 0.040,74, 4,41, 500,00, 32,00, 32,00]","69,01",[],0,0,"et,\n\n[ emitem - -—— - es pra\n\ngate (gourme..."
775,775,11,"{cálculo, desconto, alíquota, data, fiscal, no...",{rps},numero da nota\n\nprefeitura da cidade do rio ...,False,"[25, 07, 22, 14, 22, 09, 69.012, 656, 0011, 00...",15/10/2005,None,"[6,00, 0,00, 0,00, 0,00, 0,00, 35,00, 11,05, 5...","6,00",[],1,1,numero da nota\n\nprefeitura da cidade do rio ...
776,776,0,"{nota, total}","{cliente, via}",apre so\ngate gourmet ltda bandido fl nd nota ...,False,"[19833, 21941570, 103, 7, 25, 07, 22, 55, 21, ...",None,None,"[3.803,18, 243,40, 243,40]","3.803,18",[],0,0,apre so\ngate gourmet ltda bandido fl nd nota ...
777,777,22,"{cálculo, desconto, estadual, descrição, quant...",{reservado},nf-e\nnº 000.017.008\nsérie 001\n\neee\n\ndocu...,False,"[000.017, 008, 001, 3322, 0769, 0126, 5600, 11...",25/07/2022,None,"[3.803,18, 3.803,18, 3.803,18, 3.803,18, 3.803...","7,00",[],1,1,nf-e\nnº 000.017.008\nsérie 001\n\neee\n\ndocu...


In [125]:
df_nota_rotacionada = extract_text_from_pdf("C:/Users/nnico/OneDrive/Documentos/exemplo_nota_112_rotacionada.pdf",0,1)
df_nota_rotacionada

Iniciando processamento das páginas 0 a 1 de um total de 1 páginas...
Página 1/1 processada (100.00%)
Tempo total de processamento: 2.26 segundos


,Número da Página,Texto da Página,Tempo de Processamento (s)
0,1,"SLOLTIST IO OpQuUO ""IT SUAS PPASI dl Sodts O a...",2.049046


In [130]:
df_nota_rotacionada["Texto da Página"].values

array(['SLOLTIST IO OpQuUO "IT SUAS PPASI dl Sodts O a RR\n\n3 n06 Lu U0 305) mama vareta “GT dd OQUBIE ORG MY PINOOOA -\nDLOZISON 1 9O OST DE vel CHASE] OU E GONTIDLISA SP 95 G ali 1977 EU OPfRÓSRI WIOS DOQUIO 1Oj 6-5 24 SSI -\n\nao Va Oô uns aa evILLNO\n\noo\'aço\'z\n\nt erIeTaA\n\nsodnas\n\n6T0Z/20/57 *INOJ7OSUD\nZ0/81 7 hd\n\n9-SdN - VIINQUIITTI SOSIANIS 30 TVISIS VLON\nVONIZVA 30 TVdIDINNH VINVIINIIS\n-— QUIINYT 30 OIN 00 IAVOID VO VENLIIIINA\n\n'],
      dtype=object)